In [1]:
#trying to implement an rnn for next word prediction but im failing miserably at it

In [2]:
#imports
import pandas as pd
import re
from collections import Counter
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
import torch.optim as optim
import matplotlib.pyplot as plt

In [13]:
#some functions ill be using for data preprocessing
def clean_text(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]','',text)
    return text

def tokenize(text):
    return text.split()

vocab = {'<UNK>':0}

def generate_vocab(texts):
    counter = Counter()
    for text in texts:
        counter.update(tokenize(text))
    for word in counter:
        vocab[word] = len(vocab)

def encode_text(text):
    return [vocab.get(token,vocab['<UNK>']) for token in tokenize(str(text))]

def build_dataset(dataset):
    input_set = []
    output_set = []

    for sentence in dataset:
        for i in range(0,len(sentence)-1):
            input_set.append(sentence[:i+1])
            output_set.append(sentence[i+1])

    return zip(input_set,output_set)

In [14]:
#loading data
df = pd.read_csv('dataset_book.txt', encoding='latin1', sep='\t', quotechar='"', names=['text'])

df['text'] = df['text'].apply(clean_text)
generate_vocab(df.text)
df['text'] = df['text'].apply(encode_text)

train_set, test_set = train_test_split(df.text,train_size=0.8,test_size=0.2,shuffle=False)

train_set = train_set.reset_index(drop=True)
test_set = test_set.reset_index(drop=True)

train_set = build_dataset(list(train_set))
train_set = pd.DataFrame(train_set,columns=['words','prediction'])

test_set = build_dataset(list(test_set))
test_set = pd.DataFrame(test_set,columns=['words','prediction'])

In [15]:
#custom dataset class
class CustomDataset(Dataset):

    def __init__(self,df):
        self.x = df.words
        self.y = df.prediction

    def __len__(self):
        return len(self.x)

    def __getitem__(self,index):
        return torch.tensor(self.x.iloc[index], dtype=torch.long), torch.tensor(self.y.iloc[index], dtype=torch.long)

def collatefn(batch):
    text, labels = zip(*batch)
    text = pad_sequence(text,padding_value=0,batch_first=True,padding_side='left')
    return text,torch.tensor(labels)

In [16]:
#creating dataloaders
train_set = CustomDataset(train_set)
train_loader = DataLoader(train_set,batch_size=32,shuffle=False,collate_fn=collatefn)

test_set = CustomDataset(test_set)
test_loader = DataLoader(test_set, shuffle=False, batch_size=32, collate_fn=collatefn)

In [17]:
#defining the rnn
class rnn(nn.Module):

    def __init__(self,vocab_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size,120,padding_idx=0)
        nn.init.uniform_(self.embed.weight, -0.1, 0.1)
        self.rnn1 = nn.LSTM(120,500, batch_first=True)
        self.bn1 = nn.BatchNorm1d(500)
        self.rnn2 = nn.LSTM(500, 1000,batch_first=True)
        self.bn2 = nn.BatchNorm1d(1000)
        self.rnn3 = nn.LSTM(1000,2000,batch_first=True)
        self.bn3 = nn.BatchNorm1d(2000)
        self.rnn4 = nn.LSTM(2000,3000,batch_first=True)
        self.bn4 = nn.BatchNorm1d(3000)
        self.fc = nn.Linear(3000,vocab_size)

    def forward(self,X):
         out = self.embed(X)

         out, _ = self.rnn1(out)
         out = self.bn1(out.transpose(1, 2)).transpose(1, 2)

         out, _ = self.rnn2(out)
         out = self.bn2(out.transpose(1, 2)).transpose(1, 2)

         out, _ = self.rnn3(out)
         out = self.bn3(out.transpose(1, 2)).transpose(1, 2)

         out, _ = self.rnn4(out)
         out = self.bn4(out.transpose(1, 2)).transpose(1, 2)

         out = self.fc(out)
         return out

#initializing the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = rnn(len(vocab))
model = model.to(device)

#hyperparameters
n_epochs = 60
learning_rate = 0.0001
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay=0.0001)

In [18]:
#training loop
graph = {}

for epoch in range(n_epochs):
    total_loss = 0
    for batch_x, batch_y in train_loader:

        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        y_hat = model(batch_x)

        y_hat = y_hat[:,-1,:]

        l = loss(y_hat,batch_y)

        l.backward()

        optimizer.step()

        optimizer.zero_grad()

        total_loss += l.item() * batch_x.size(0)

    avg_loss = total_loss / len(train_loader.dataset)

    graph[epoch+1] = avg_loss

    #if (epoch+1)%10 == 0:
    print(f'For Epoch {epoch+1}: Loss = {avg_loss}')

#learning curve
plt.title('Loss Vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(graph.keys(),graph.values())

KeyboardInterrupt: 

In [ ]:
#model evaluation for training set
model.eval()

total_correct = 0
total = 0

for batch_x, batch_y in train_loader:

    batch_x = batch_x.to(device)
    batch_y = batch_y.to(device)

    prediction = model(batch_x)

    prediction = prediction[:,-1,:]

    _ , output = torch.max(prediction, 1)

    total_correct += (output==batch_y).sum().item()

    total += batch_y.size(0)


accuracy = total_correct / total * 100
print(f'Training Set Accuracy: {accuracy:.2f}%')

Training Set Accuracy: 17.18%


In [ ]:
#model evaluation for test set
model.eval()

total_correct = 0
total = 0

for batch_x, batch_y in test_loader:

    batch_x = batch_x.to(device)
    batch_y = batch_y.to(device)

    prediction = model(batch_x)

    prediction = prediction[:,-1,:]

    _ , output = torch.max(prediction, 1)

    total_correct += (output==batch_y).sum().item()

    total += batch_y.size(0)


accuracy = total_correct / total * 100
print(f'Test Set Accuracy: {accuracy:.2f}%')

Test Set Accuracy: 7.12%
